In [2]:
import sys
import glob
import json
import torch
import pandas as pd
sys.path.append('/home/long8v/BERT-NER')
from bert import Ner

# Data Loading

In [22]:
path = '/home/long8v/ICDAR-2019-SROIE/task3/data/test_dict.pth'
data_dict = torch.load(path)

# inference

In [23]:
import re
import pickle
from transformers import *

In [24]:
model = Ner('/home/long8v/sroie_data/raw_bert_replace_special_token')

In [25]:
tokenizer = model.tokenizer
tokenizer.vocab_size

28996

In [26]:
# pretrained_weights = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
# print(tokenizer.vocab_size)
re_int = re.compile('\d+')
re_float = re.compile('(\d+\.\d+)')
re_percent = re.compile('(\d+.?\d+%)')
re_date = re.compile('(\d{2}[/-]\d{2}[/-]\d{2,4})')

In [27]:
re_dict = {re_float:'float', re_percent:'percent', re_date:'date', re_int:'int'}

In [28]:
def re_text(text):
    for key, value in re_dict.items():
        text = key.sub(value, text)
    return text

In [29]:
def get_tokenized_word(text):
    token_word = tokenizer.tokenize(text)
    return token_word

In [30]:
def preprocess(text):
    text = re_text(text)
    return text

In [31]:
preprocess_data = {}
for key, value in data_dict.items():
    preprocess_data[key] = preprocess(value)

In [37]:
len(tokenizer.tokenize(value))

349

In [38]:
result_data = {}
for key, value in preprocess_data.items():
    result_data[key] = model.predict(value)

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
sample_result = [{'word': 'hello', 'tag': 'B-COMPANY', 'confidence': 0.4645322561264038}]

In [33]:
from collections import defaultdict

In [45]:
def get_result_json(result_list):
    result_json = defaultdict(str)
    for sample in result_list:
        tag = sample['tag'].lower()
        word = sample['word']
        word = word.replace('##', '')
        if tag != 'O':
            result_json[tag.split('-')[1]] += ' '+word 
    return dict(result_json)

In [153]:
(get_result_json(sample_result))

{'company': 'hello'}

In [40]:
path = '/home/long8v/docrv2_sroie/submission/SROIE_example_t3'
for key, value in result_data.items():
    with open('{}/{}.txt'.format(path, key), 'w') as f:
        json.dump(value, f)

In [43]:
result_data = {0:1}